<h3>Tweet extraction</h3>

In [178]:
#API KEYS
consumer_key = "FzQhcfBvKlRwlzLPusKrrSY6P"
consumer_secret = "ITDtL5vsBgyC3N6YrT7sjh8t2WNFNwFeNojLlqmQPH1iIlEaaa"
access_token = "1431117962350448643-59wKX0XbMqwFaI9KDetGi0RlFQKU2G"
access_token_secret = "XIm6pu4mgVX6ocrEVv7WIC9cBKLRnpMFJy9vjn3wRKjFG"

In [179]:
#package imports
import tweepy
import ssl
import pandas as pd
import html5lib
import requests
from bs4 import BeautifulSoup
import lxml
import sklearn
import numpy as np

In [180]:
#Authorisation
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [181]:
#Fetching recent tweet id
name = "samlutd"
statuses = api.user_timeline(screen_name = "samlutd",count=1)
for status in statuses:
    tweet_id = status.id    
print(tweet_id)

1489483507252355072


In [221]:
#Recent tweet comment extraction
replies=[]
users = []
for tweet in tweepy.Cursor(api.search_tweets,q='to:'+name, result_type='recent').items(10):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        replies.append(tweet.text)        
        users.append(api.get_status(tweet._json['id']).user.screen_name)


In [242]:
replies.append("SHUT UP MOTHERFUCKER")
users.append("Drake")

In [243]:
interaction = {"comments":replies,"user":users}

In [244]:
replies_df = pd.DataFrame(replies,columns = ["comments"])
results = pd.DataFrame.from_dict(interaction)

In [245]:
results

,comments,user
0,@samlutd Let's see,rehanshhirani
1,@samlutd @Cricthoughts03 I hope so,noplasticinsea
2,@samlutd @SaHiListir3dtbh Hear me out sam\n\nT...,Cricthoughts03
3,@samlutd @ftbltaban Why not rashy in place of ...,XVenom07
4,@samlutd @SaHiListir3dtbh That's what I wrote ...,Cricthoughts03
5,@samlutd For just fa Cup right,ftbltaban
6,@samlutd @Cricthoughts03 Like they are gonna d...,SaHiListir3dtbh
7,@samlutd Not jones and Maguire 😅🤦🏿‍♂️,Ibzi10_
8,@samlutd age:19\nheight:5'9\ncountry:australia...,raikrrraik
9,@samlutd https://t.co/2eyITxqN2o,MelbourneBlokee


<h1>Testing Tweet</h1>

In [270]:
train = pd.read_csv("train.csv")
test = replies_df

In [271]:
sum(train["label"] == 0)
sum(train["label"] == 1)

train.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [272]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [273]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [274]:
train_tweet = clean_tweets(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)

train["clean_tweet"] = train_tweet


In [275]:
test_tweet = clean_tweets(test["comments"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet


In [276]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.label.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(train.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

In [296]:
from sklearn.feature_extraction.text import CountVectorizer

documents = ["This is Import Data's Youtube channel",
             "Data science is my passion and it is fun!",
             "Please subscribe to my channel" ]

# initializing the countvectorizer
vectorizer = CountVectorizer()

# tokenize and make the document into a matrix
document_term_matrix = vectorizer.fit_transform(documents)

# check the result
pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names())


C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,and,channel,data,fuck,fun,import,is,it,my,passion,please,science,subscribe,this,to,youtube
0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,0,1
1,1,0,1,0,1,0,2,1,1,1,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [297]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [298]:
x_test_vec = vectorizer.transform(test.clean_tweet.values)

In [299]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)


In [300]:
y_pred_svm

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [291]:
test

,comments,clean_tweet
0,@samlutd Let's see,lets see
1,@samlutd @Cricthoughts03 I hope so,i hope so
2,@samlutd @SaHiListir3dtbh Hear me out sam\n\nT...,hear me out samtest cricket is the only format...
3,@samlutd @ftbltaban Why not rashy in place of ...,why not rashy in place of elanga rashy needs s...
4,@samlutd @SaHiListir3dtbh That's what I wrote ...,thats what i wrote matetest cricket is the onl...
5,@samlutd For just fa Cup right,for just fa cup right
6,@samlutd @Cricthoughts03 Like they are gonna d...,like they are gonna do this after seeing my tweet
7,@samlutd Not jones and Maguire 😅🤦🏿‍♂️,not jones and maguire
8,@samlutd age:19\nheight:5'9\ncountry:australia...,age19height59countryaustraliafave playernnyrel...
9,@samlutd https://t.co/2eyITxqN2o,


In [269]:
test.clean_tweet.values

array(['lets see', 'i hope so',
       'hear me out samtest cricket is the only format that is relevant and should be played',
       'why not rashy in place of elanga rashy needs some form imo',
       'thats what i wrote matetest cricket is the only thing keeping international cricket relevant',
       'for just fa cup right',
       'like they are gonna do this after seeing my tweet',
       'not jones and maguire',
       'age19height59countryaustraliafave playernnyrelationship statusinglenameamuel',
       '', 'shut up motherfucker'], dtype=object)